In [1]:
import pandas as pd
import re
import requests
import unicodedata
from bs4 import BeautifulSoup



In [2]:
from helpers import *

## Version 1:

### Extract 10-K text and semgment into sections designated by table of contents for a specific filing document

In [3]:
parsed_tenk_text_dict = get_text_dict(r"https://www.sec.gov/Archives/edgar/data/1318605/000119312511054847/0001193125-11-054847.txt", '10-K')

In [5]:
parsed_tenk_text_dict.keys()
# parsed_tenk_text_dict['Controls and Procedures']

dict_keys(['Business', 'Risk Factors', 'Unresolved Staff Comments', 'Properties', 'Legal Proceedings', 'Reserved', 'Market for Registrant\x92s Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities', 'Selected Financial Data', 'Management\x92s Discussion and Analysis of Financial Condition and Results of Operations', 'Quantitative and Qualitative Disclosures About Market Risk', 'Financial Statements and Supplementary Data', 'Changes in and Disagreements with Accountants on Accounting and Financial Disclosure', 'Controls and Procedures', 'Other Information', 'Directors, Executive Officers and Corporate Governance', 'Executive Compensation', 'Security Ownership of Certain Beneficial Owners and Management and Related Stockholder Matters', 'Certain Relationships and Related Transactions, and Director Independence', 'Principal Accountant Fees and Services', 'Exhibits and Financial Statement Schedules '])

In [6]:
parsed_tenk_text_dict['Market for Registrant\x92s Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities']

'Our common stock has traded on The NASDAQ Global Select Market under the symbol TSLA since it began trading on June 29, 2010. Our initial public offering was priced at $17.00 per share on June 28, 2010. The following table sets forth, for the time period indicated, the high and low closing sales price of our common stock as reported on The NASDAQ Global Select Market. 2010 High Low Second Quarter (from June 29, 2010) $ 23.89 $ 23.83 Third Quarter 21.98 15.80 Fourth Quarter 35.47 20.05 As of January 31, 2011, there were 365 holders of record of our common stock. A substantially greater number of holders of our common stock are street name or beneficial holders, whose shares are held by banks, brokers and other financial institutions. Dividend Policy We have never declared or paid cash dividends on our common or convertible preferred stock. We currently do not anticipate paying any cash dividends in the foreseeable future. Any future determination to declare cash dividends will be made 

In [ ]:
testing_eightk = get_text_dict(r'https://www.sec.gov/Archives/edgar/data/1318605/000156459021012981/0001564590-21-012981.txt', '8-K')

## We have text organized into these sections:

In [ ]:
for k in parsed_tenk_text_dict.keys():
    print(k)

## We also want to do the same with 10-Q forms for quarterly reports. 
### Part of the analysis will be comparing 10-Qs with past 10-Qs

## Cosine Similarity

Will compare the cosine similarity from previous doc sections to current sections

### Testing nlp pipeline

In [ ]:
import unicodedata

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode()

input_string = remove_accents(parsed_tenk_text_dict['CONTROLS AND PROCEDURES '])

In [ ]:
input_string

### Tokenizing

In [ ]:
from nltk.tokenize import sent_tokenize

sent_tokens = sent_tokenize(input_string)

sent_tokens

### nltk.tokenize.word_tokenize

In [ ]:
from nltk.tokenize import word_tokenize

tokens = [word_tokenize(sent) for sent in sent_tokens]

list(enumerate(tokens))

In [ ]:
## make lowercase

import string

tokens_lower = [[word.lower() for word in sent]
                 for sent in tokens]

## Filter stopwords and punctuation

### will need to go back and create custom stopwords list

In [ ]:
from nltk.corpus import stopwords

stopwords_ = set(stopwords.words('english'))
punctuation_ = set(string.punctuation)
print("--- punctuation: {}".format(string.punctuation))

def filter_tokens(sent):
    return([w for w in sent if not w in stopwords_ and not w in punctuation_])

tokens_filtered = list(map(filter_tokens, tokens_lower))

for sent in tokens_filtered:
    print("--- sentence tokens: {}".format(sent))

In [ ]:
tokens_filtered

## Stemming and lemmatization

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

sentence_number = 0

stemmer_porter = PorterStemmer()
tokens_stemporter = [list(map(stemmer_porter.stem, sent)) for sent in tokens_filtered]
print("--- sentence tokens (porter): {}".format(tokens_stemporter[sentence_number]))

stemmer_snowball = SnowballStemmer('english')
tokens_stemsnowball = [list(map(stemmer_snowball.stem, sent)) for sent in tokens_filtered]
print("--- sentence tokens (snowball): {}".format(tokens_stemsnowball[sentence_number]))

lemmatizer = WordNetLemmatizer()
tokens_lemmatize = [list(map(lemmatizer.lemmatize, sent)) for sent in tokens_filtered]
print("--- sentence tokens (lemmatize): {}".format(tokens_lemmatize[sentence_number]))